# CS 440 Project on Rubik's Cube solving with Reinforcement Learning

_by David Edwards, December 12, 2017_

## Introduction

My son has long been fascinated by Rubik's Cubes, and yet seems to be completely incapable of solving them.  I decided to try out his 2x2 cube.  <img src="2x2.jpg">

I grew frustrated and failed even faster, and rather than research the existing solutions that were online, I decided to throw one of our algorithms and some brute force at it.  

When the proposal time came around, I realized that the Reinforcement Learning process was the hardest for me to understand, and yet when I understood it, I felt like I really understood it, so wanted to go with that approach.

## Methods

I foresaw three main problems with the task I had chosen.

1. Representing the state of the puzzle
2. Creating the MakeMoves function
3. Ensuring it runs in memory

Here was my approach:


I tried using some Python text graphics libraries to display the sides in their actual color, thinking that would make things easier.  I tried using [termcolor](https://pypi.python.org/pypi/termcolor) but found that the colors didn't match the actual Rubik's cube colors of Red, Blue, Yellow, Orange, White and Green.  There were existing Rubik's Cube Python libraries like [PyCuber](https://github.com/adrianliaw/PyCuber) and this fancy [interactive cube](https://jakevdp.github.io/blog/2012/11/26/3d-interactive-rubiks-cube-in-python/) but I wanted to do as much as possible from scratch.  So, I made the cube a nested list of list of lists.  The lowest level lists represented the top two colors of a side, the next the bottom two colors.  Then, the next level of lists was one whole face of the cube, and the final level was all six sides of the cube.

In [32]:
completeState = [[["Red", "Red"],["Red","Red"]],[["Blue", "Blue"],["Blue","Blue"]],[["Yellow", "Yellow"],["Yellow","Yellow"]],[["Orange", "Orange"],["Orange","Orange"]],[["White", "White"],["White","White"]],[["Green", "Green"],["Green","Green"]]]
completeState

[[['Red', 'Red'], ['Red', 'Red']],
 [['Blue', 'Blue'], ['Blue', 'Blue']],
 [['Yellow', 'Yellow'], ['Yellow', 'Yellow']],
 [['Orange', 'Orange'], ['Orange', 'Orange']],
 [['White', 'White'], ['White', 'White']],
 [['Green', 'Green'], ['Green', 'Green']]]

Then, I needed to figure out how to display a cube in a state I could grasp with a little bit of confidence.  I created these functions to help with that.  The initial variables are my attempt at using something like constants in Python, which doesn't seem to really support them.  But, I wanted the logic for these to make sense to me later, so I just created these values, even though if I wasn't careful, they could be overwritten later.

One bonus was that each color used in a cube had different first letters, so I could just display the first letter, making it slightly easier to read.

In [72]:
import copy as copy
import numpy as np
import random
import time

FRONT = 1
LEFT = 0
TOP = 2
BOTTOM = 3
RIGHT = 4
BACK = 5

def printFirstLetters(pair):
    '''
    Prints the first letter of a given pair.
    :param pair: list containing two colors'''
    return("{}\t{}\t".format(pair[0][:1],pair[1][:1]))


def printState(state):
    '''
    Prints a 2x2 Rubik's cube state.  
    :param state: list of list of lists representing Rubik's cube state
    :return: prints out the state nicely.
    '''
    print("\t\t", printFirstLetters(state[TOP][0]))
    print("\t\t", printFirstLetters(state[TOP][1]))
    print(printFirstLetters(state[LEFT][0]), printFirstLetters(state[FRONT][0]),printFirstLetters(state[RIGHT][0]),printFirstLetters(state[BACK][0]))
    print(printFirstLetters(state[LEFT][1]), printFirstLetters(state[FRONT][1]),printFirstLetters(state[RIGHT][1]),printFirstLetters(state[BACK][1]))
    print("\t\t", printFirstLetters(state[BOTTOM][0]))
    print("\t\t", printFirstLetters(state[BOTTOM][1]))
    print("---------------------------------------------------------")


In [19]:
printState(completeState)

		 Y	Y	
		 Y	Y	
R	R	 B	B	 W	W	 G	G	
R	R	 B	B	 W	W	 G	G	
		 O	O	
		 O	O	
---------------------------------------------------------


This represents my "unfolding" of a cube.  In this case, the blue face was the one facing "me", the red to the left, the yellow on top, orange on the bottom, white to my right, green behind.  The line underneath is a separator, to make it easier to visualize when I'm displaying tens or hundreds of them.

Next, I had to make a validMove and makeMove function.

In [21]:
def validMoves(state):
    '''
    Returns a list of lists representing valid 2x2 rubik's cube moves from the given state.
    Move Rotations courtesy: http://www.rubiksplace.com/move-notations/
    :param state: list of lists representing tower of hanoi state
    :return: list of lists representing valid tower of hanoi moves from the given state.
    '''

    # evaluate if we need the front and back moves
    # validStates = ["U", "U'", "D", "D'", "R", "R'", "L", "L'", "F", "F'", "B", "B'"]

    validStates = ["U", "Uprime", "D", "Dprime", "R", "Rprime", "L", "Lprime"]

    return validStates


def makeMove(state, move, printMoves=False):
    '''
    Takes a move and makes it 2x2 rubik's cube
    :param state: list of lists representing cube
    :param move: possible rubik's cube move
    :return:the state after the move was made
    '''

    if move == "U":
        state[LEFT][0], state[FRONT][0], state[RIGHT][0], state[BACK][0] = state[FRONT][0], state[RIGHT][0], state[BACK][0], state[LEFT][0]
    if move == "Uprime":
        state[FRONT][0], state[RIGHT][0], state[BACK][0], state[LEFT][0] = state[LEFT][0], state[FRONT][0], state[RIGHT][0], state[BACK][0]
    if move == "D":
        state[LEFT][1], state[FRONT][1], state[RIGHT][1], state[BACK][1] = state[FRONT][1], state[RIGHT][1], state[BACK][1], state[LEFT][1]
    if move == "Dprime":
        state[FRONT][1], state[RIGHT][1], state[BACK][1], state[LEFT][1] = state[LEFT][1], state[FRONT][1], state[RIGHT][1], state[BACK][1]
    if move == "R":
        state[TOP][0][1], state[TOP][1][1], state[FRONT][0][1], state[FRONT][1][1], state[BOTTOM][0][1], state[BOTTOM][1][1], state[BACK][0][1], state[BACK][1][1] = state[FRONT][0][1], state[FRONT][1][1], state[BOTTOM][0][1], state[BOTTOM][1][1], state[BACK][0][1], state[BACK][1][1],state[TOP][0][1], state[TOP][1][1]
    if move == "Rprime":
        state[FRONT][0][1], state[FRONT][1][1], state[BOTTOM][0][1], state[BOTTOM][1][1], state[BACK][0][1], state[BACK][1][1], state[TOP][0][1], state[TOP][1][1] = state[TOP][0][1], state[TOP][1][1], state[FRONT][0][1], state[FRONT][1][1], state[BOTTOM][0][1], state[BOTTOM][1][1], state[BACK][0][1], state[BACK][1][1]
    if move == "L":
        state[TOP][0][0], state[TOP][1][0], state[FRONT][0][0], state[FRONT][1][0], state[BOTTOM][0][0], state[BOTTOM][1][0], state[BACK][0][0], state[BACK][1][0] = state[FRONT][0][0], state[FRONT][1][0], state[BOTTOM][0][0], state[BOTTOM][1][0], state[BACK][0][0], state[BACK][1][0],state[TOP][0][0], state[TOP][1][0]
    if move == "Lprime":
        state[FRONT][0][0], state[FRONT][1][0], state[BOTTOM][0][0], state[BOTTOM][1][0], state[BACK][0][0], state[BACK][1][0], state[TOP][0][0], state[TOP][1][0] = state[TOP][0][0], state[TOP][1][0], state[FRONT][0][0], state[FRONT][1][0], state[BOTTOM][0][0], state[BOTTOM][1][0], state[BACK][0][0], state[BACK][1][0]

    if printMoves:
        printState(state)

    return state

I have gone through several iterations of the validMoves function, and I'm not convinced this is the correct one.  I use the move notation found [here](http://www.rubiksplace.com/move-notations/), where U is move the top level clockwise, and Uprime moves the top level counter-clockwise.  Same with Down, Right, and Left.  Some had Back as well.  I may simplify these to reduce the size of the Q table.  Up, Up, Up is the same as UpPrime, so I want to minimize the total number of State,ValidMove pairs in the Q table, due to the size of the Rubik's Cube which has [3674160](https://www.therubikzone.com/number-of-combinations/) states.  I would rather have that be multiplied by 4 or 5 valid moves, rather than 8 or 10.

Regardless, let's make sure the makeMove works:

In [36]:
completeState = [[["Red", "Red"],["Red","Red"]],[["Blue", "Blue"],["Blue","Blue"]],[["Yellow", "Yellow"],["Yellow","Yellow"]],[["Orange", "Orange"],["Orange","Orange"]],[["White", "White"],["White","White"]],[["Green", "Green"],["Green","Green"]]]
print("Complete State:")
printState(completeState)
print("Left side rotation:")
x = makeMove(completeState, "L", True)

Complete State:
		 Y	Y	
		 Y	Y	
R	R	 B	B	 W	W	 G	G	
R	R	 B	B	 W	W	 G	G	
		 O	O	
		 O	O	
---------------------------------------------------------
Left side rotation:
		 B	Y	
		 B	Y	
R	R	 O	B	 W	W	 Y	G	
R	R	 O	B	 W	W	 Y	G	
		 G	O	
		 G	O	
---------------------------------------------------------


Here you can see we moved the left side of the cube, which pivots the Blue front to the top, the Orange bottom to the front, Green back to the bottom, and Yellow top to the back.  If we do this two more times, it should be back to the start state:

In [37]:
for i in range(3):
    makeMove(completeState,"L",True)


		 O	Y	
		 O	Y	
R	R	 G	B	 W	W	 B	G	
R	R	 G	B	 W	W	 B	G	
		 Y	O	
		 Y	O	
---------------------------------------------------------
		 G	Y	
		 G	Y	
R	R	 Y	B	 W	W	 O	G	
R	R	 Y	B	 W	W	 O	G	
		 B	O	
		 B	O	
---------------------------------------------------------
		 Y	Y	
		 Y	Y	
R	R	 B	B	 W	W	 G	G	
R	R	 B	B	 W	W	 G	G	
		 O	O	
		 O	O	
---------------------------------------------------------


In addition, performing a move, and then it's prime should return to the original state:

Then, we need to determine if a move consists of a winner.  You'll see remnants of my first attempt, which basically checked to see if the current state was equal to the completeState above.  However, I realized that misses out on 5 other complete states, when the cube faces are all of the same color, but the colors aren't in the same position as they initially were.  I fixed that.

In [54]:
def winner(state):
    '''
    Determines if a winning state occured
    :param state: list of lists representing tower of hanoi state
    :return: True if winning state, False otherwise.
    '''
    completeState = [[["Red", "Red"], ["Red", "Red"]], [["Blue", "Blue"], ["Blue", "Blue"]],
                     [["Yellow", "Yellow"], ["Yellow", "Yellow"]], [["Orange", "Orange"], ["Orange", "Orange"]],
                     [["White", "White"], ["White", "White"]], [["Green", "Green"], ["Green", "Green"]]]

    return faceComplete(state[LEFT]) and faceComplete(state[FRONT]) and faceComplete(state[TOP]) and faceComplete(state[BOTTOM]) and faceComplete(state[RIGHT]) and faceComplete(state[BACK]) 

    # return state == completeState

def faceComplete(face):
    return (face[0][0]== face[0][1] == face[1][0] == face[1][1])

In [56]:
completeState = [[["Red", "Red"],["Red","Red"]],[["Blue", "Blue"],["Blue","Blue"]],[["Yellow", "Yellow"],["Yellow","Yellow"]],[["Orange", "Orange"],["Orange","Orange"]],[["White", "White"],["White","White"]],[["Green", "Green"],["Green","Green"]]]
print(winner(completeState))
makeMove(completeState,'R',True)
print(winner(completeState))
makeMove(completeState,'Rprime',True)
print(winner(completeState))

True
		 Y	B	
		 Y	B	
R	R	 B	O	 W	W	 G	Y	
R	R	 B	O	 W	W	 G	Y	
		 O	G	
		 O	G	
---------------------------------------------------------
False
		 Y	Y	
		 Y	Y	
R	R	 B	B	 W	W	 G	G	
R	R	 B	B	 W	W	 G	G	
		 O	O	
		 O	O	
---------------------------------------------------------
True


In [40]:
completeState = [[["Red", "Red"],["Red","Red"]],[["Blue", "Blue"],["Blue","Blue"]],[["Yellow", "Yellow"],["Yellow","Yellow"]],[["Orange", "Orange"],["Orange","Orange"]],[["White", "White"],["White","White"]],[["Green", "Green"],["Green","Green"]]]
printState(completeState)
_ = makeMove(completeState,"R",True)
_ = makeMove(completeState,"Rprime", True)

		 Y	Y	
		 Y	Y	
R	R	 B	B	 W	W	 G	G	
R	R	 B	B	 W	W	 G	G	
		 O	O	
		 O	O	
---------------------------------------------------------
		 Y	B	
		 Y	B	
R	R	 B	O	 W	W	 G	Y	
R	R	 B	O	 W	W	 G	Y	
		 O	G	
		 O	G	
---------------------------------------------------------
		 Y	Y	
		 Y	Y	
R	R	 B	B	 W	W	 G	G	
R	R	 B	B	 W	W	 G	G	
		 O	O	
		 O	O	
---------------------------------------------------------


We also need to be able to get a Tuple from the state,move pair which, given that a state is a three-deep list of lists, that was made a little more difficult.  Here's what I came up with:

In [60]:
def getTuple(state, move):
    '''
    Need immutable type for key to dictionary
    :param state: list of lists representing rubik's cube state
    :return: tuple representation of the state
    '''
    superTuple = tuple(tuple(tuple(s[0])+tuple(s[1])) for s in state)
    return (superTuple, move)

In [61]:
completeState = [[["Red", "Red"],["Red","Red"]],[["Blue", "Blue"],["Blue","Blue"]],[["Yellow", "Yellow"],["Yellow","Yellow"]],[["Orange", "Orange"],["Orange","Orange"]],[["White", "White"],["White","White"]],[["Green", "Green"],["Green","Green"]]]
getTuple(completeState,'R')

((('Red', 'Red', 'Red', 'Red'),
  ('Blue', 'Blue', 'Blue', 'Blue'),
  ('Yellow', 'Yellow', 'Yellow', 'Yellow'),
  ('Orange', 'Orange', 'Orange', 'Orange'),
  ('White', 'White', 'White', 'White'),
  ('Green', 'Green', 'Green', 'Green')),
 'R')

That's not attractive, but it does appear to be a valid Tuple.  The 'R' at the very end represents a Right side turn.

Now we have the basics to start filling our Q.  I'm using the same trainQ and epsilonGreedy from [Assignment 5](https://github.com/doofusdavid/CS440-A5).  

In [75]:
def epsilonGreedy(epsilon, Q, state, validMovesF):
    '''
    Makes either a random move, or tries the move which Q indicates is the best.
    :param epsilon: A decreasing number representing the level of randomness
    :param Q: Dictionary of state,move - value pairs, with the higher values being better moves
    :param state: list of lists representing tower of hanoi state
    :param validMovesF: function returning valid moves
    :return:
    '''
    goodMoves = validMovesF(state)
    if np.random.uniform() < epsilon:
        # Random Move
        return random.choice(goodMoves)
    else:
        # Greedy Move
        Qs = np.array([Q.get(getTuple(state,m), 0.0) for m in goodMoves])
        return goodMoves[np.argmax(Qs)]


def trainQ(startState, nRepetitions, learningRate, epsilonDecayFactor, validMovesF, makeMoveF):
    '''
    Creates and fills a dictionary, Q, representing the (state,move) - value pairs which, if followed
    should create the shortest path to the solution.
    :param nRepetitions: how many times to iterate through.  Higher numbers would generate more accurate results
    :param learningRate: How much to adjust the value part of the dictionary
    :param epsilonDecayFactor: how quickly to reduce the random factor.
    :param validMovesF: function returning valid moves of a state
    :param makeMoveF: function making a move on a state
    :return: the dictionary, Q, and a list containing the number of steps it took per iteration to find the goal state
    '''
    maxGames = nRepetitions
    rho = learningRate
    epsilonDecayRate = epsilonDecayFactor
    epsilon = 1.0
    Q = {}
    stepList = []
    # show the moves while debuggin
    showMoves = False

    for nGames in range(maxGames):
        # if nGames % 10 == 0: print(".", end="")
        # if nGames % 100 == 0: print("Q length: ", len(Q))
        # reduce the randomness every pass
        epsilon *= epsilonDecayRate
        step = 0
        # hardcoded start state
        state = startState
        done = False

        while not done:
            #if step % 100 == 0: print(".", end="")
            #if step % 1000 == 0: print("Q length: ", len(Q))
            step += 1
            # grab either a random or best of the known moves
            move = epsilonGreedy(epsilon, Q, state, validMovesF)

            # we don't want to change state directly, and because state is a list of lists, need to do a
            # deepcopy on it, then make the move
            stateNew = copy.deepcopy(state)
            makeMoveF(stateNew, move)

            # if we haven't encountered this state,move combo, add it to Q
            if getTuple(state, move) not in Q:
                Q[getTuple(state, move)] = 0.0  # Initial Q value for new state, move

            # print if debugging
            if showMoves:
                printState(stateNew)
            if winner(stateNew):
                # We won!  backfill Q
                # print('End State, we won!')
                Q[getTuple(state, move)] = -1.0
                done = True
                # we're keeping a list of the number of steps it took for each winning solution, so add it here.
                stepList.append(step)

            # update the Q which led us here using the learning factor, and the difference between the current state
            # and the old state
            if step > 1:
                Q[getTuple(stateOld, moveOld)] += rho * (-1 + Q[getTuple(state, move)] - Q[getTuple(stateOld, moveOld)])
                #print("Q[",getTuple(stateOld, moveOld),"]: ",Q[getTuple(stateOld, moveOld)])
            # Store the current state, move so we can access it for the next Q update
            stateOld, moveOld = state, move
            state = stateNew

    return Q, stepList


def testQ(Q, initialState, maxSteps, validMovesF, makeMoveF):
    '''
    Using the dictionary Q, and the initial state of the game, traverse and return the best path.
    :param Q: dictionary representing the (state,move) - value pairs which, if followed should create the shortest path to the solution.
    :param maxSteps: The number of steps to attempt before giving up.
    :param validMovesF: function returning valid moves of a state
    :param makeMoveF: function making a move on a state
    :return: list containing the states from start to finish
    '''
    #state = [[["Red", "Red"],["Red","Red"]],[["Blue", "Blue"],["Blue","Blue"]],[["Yellow", "Yellow"],["Yellow","Yellow"]],[["Orange", "Orange"],["Orange","Orange"]],[["White", "White"],["White","White"]],[["Green", "Green"],["Green","Green"]]]
    state = initialState

    statePath = []
    movePath = []
    movePath.append("Initial")
    statePath.append(state)

    for i in range(maxSteps):
        if winner(state):
            return statePath, movePath
        goodMoves = validMovesF(state)
        Qs = np.array([Q.get(getTuple(state, m), -1000.0) for m in goodMoves])
        move = goodMoves[np.argmax(Qs)]
        movePath.append(move)
        nextState = copy.deepcopy(state)
        makeMoveF(nextState, move)
        statePath.append(nextState)
        state = nextState

    return "No path found"

There are a few changes which I should address.

1. For the Towers of Hanoi, the ideal solution was 7 moves, but for a rubik's cube, it could be much larger.  I had to place some code to display some feedback as it was executing.  In fact, the first time I ran this, it ended up running for 3 straight days over a weekend on a computer at work.  That was when I discovered a few bugs.  I had defined `winner` such that a face with the top two colors matching, and the bottom two colors matching, even if they weren't all the same color was `True.`  This resulted in a trainQ that never completed.  
2. The possible length of Q (29393280, which I got from multiplying the possible states above, by 8 different moves) resulted in a great many Q values that weren't traversed yet.  During testQ, this line: 

`Qs = np.array([Q.get(getTuple(state, m), -1000.0) for m in goodMoves])` 

used to be 

`Qs = np.array([Q.get(getTuple(state, m), 0.0) for m in goodMoves])`  

This would return a 0.0 for a number of possible moves (any which weren't tested during the trainQ function), and given that the next line did an np.argmax on those values, the 0.0 was often larger than other Q values which actually had been traversed.  I changed the "default" value if that Q value didn't exist to be -1000.0, thinking that anything larger than that would be preferable.  This seems to work out pretty well.

## Results

Using this, I needed to figure out a way to test it.  My first thought was to create a random cube, but then I remembered a problem with that in an earlier assignement.  A puzzle can't necessarily be solved if it was completely randomly shuffled.  Randomly shuffling a Rubik's cube would be akin to peeling off the stickers and putting them in random places.  It's possible that there would be a solution, but also possible that a corner which has three faces could have all three faces Red, which couldn't be solved.  So, I created a shuffled cube, starting from the solved state.

In [76]:
completeState = [[["Red", "Red"],["Red","Red"]],[["Blue", "Blue"],["Blue","Blue"]],[["Yellow", "Yellow"],["Yellow","Yellow"]],[["Orange", "Orange"],["Orange","Orange"]],[["White", "White"],["White","White"]],[["Green", "Green"],["Green","Green"]]]
newstate = completeState
for i in range(1):
    move = random.choice(validMoves(newstate))
    print("Move ", i, " was: ", move)
    newstate = makeMove(newstate,move)


printState(newstate)
startTime = time.time()
Q, steps = trainQ(newstate, 500, 0.5, 0.7, validMoves, makeMove)
endTime = time.time()
print(steps)
path, moveList = testQ(Q, newstate, 20000, validMoves, makeMove)

print("Training took: ", endTime-startTime, " seconds.")
print("Mean of solution length: ", np.mean(steps))
if path == "No path found":
    print(path)
else:
    for i in range(len(path)):
        printState(path[i])
        print("Move: ", moveList[i])


Move  0  was:  D
		 Y	Y	
		 Y	Y	
R	R	 B	B	 W	W	 G	G	
B	B	 W	W	 G	G	 R	R	
		 O	O	
		 O	O	
---------------------------------------------------------
[1, 233, 1, 5, 1, 18139, 46787, 7, 5, 13, 129935, 44731, 3, 51975, 33, 41043, 175, 4525, 73, 8467, 36379, 98433, 569, 3, 25, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

Below is a sample run, with 30 random moves.  I made it a python comment to avoid negatively affecting word count.  

It took 9 minutes to run, returning a mean solution length of 721.  I would have liked to see that closer to 30, as that was the total number of moves to get the puzzle to that state.  Note that it's quite possible, and in this case it's true, that the solution was considerably less than the 30 initial moves.  Some of the moves may reduce the randomness...an Up followed by an UpPrime, for example, is effectively no state change.  In this case it only took 6 moves to complete the puzzle and return it to it's solved state.  

In [ ]:
'''
Move  0  was:  Rprime
Move  1  was:  D
Move  2  was:  Dprime
Move  3  was:  U
Move  4  was:  Dprime
Move  5  was:  R
Move  6  was:  R
Move  7  was:  L
Move  8  was:  U
Move  9  was:  D
Move  10  was:  L
Move  11  was:  Dprime
Move  12  was:  D
Move  13  was:  U
Move  14  was:  R
Move  15  was:  R
Move  16  was:  U
Move  17  was:  Rprime
Move  18  was:  L
Move  19  was:  Uprime
Move  20  was:  R
Move  21  was:  R
Move  22  was:  U
Move  23  was:  U
Move  24  was:  Dprime
Move  25  was:  U
Move  26  was:  U
Move  27  was:  Rprime
Move  28  was:  Rprime
Move  29  was:  Rprime
		 W	W	
		 W	W	
G	B	 Y	O	 B	G	 O	Y	
B	G	 O	Y	 G	B	 Y	O	
		 R	R	
		 R	R	
-------------------------------
[1464, 3566, 31432, 22586, 780, 26884, 25242, 2184, 17126, 89074, 5056, 68990, 89022, 5658, 51930, 6966, 40084, 15926, 5924, 25488, 21170, 50222, 8052, 9988, 41192, 30996, 2876, 12392, 1368, 12942, 9066, 26418, 1966, 3070, 1644, 15062, 5352, 7584, 2264, 264, 22226, 40060, 4802, 3640, 774, 14058, 5412, 314, 18678, 7712, 1194, 14358, 2258, 322, 10376, 14334, 2006, 474, 6344, 15752, 14922, 598, 13518, 64, 312, 5564, 16178, 10152, 10, 4262, 33786, 760, 4930, 712, 6480, 6306, 1384, 11024, 31292, 9234, 2544, 280, 1598, 6912, 7490, 2836, 294, 12, 10210, 120, 18384, 11936, 3890, 356, 54, 20162, 8892, 4180, 19754, 24562, 6520, 18902, 14620, 4524, 13672, 628, 16838, 6886, 10838, 1222, 11722, 6774, 6300, 9200, 7132, 3984, 2216, 12, 9198, 42362, 410, 8, 1184, 3700, 2996, 172, 11564, 72778, 24098, 14570, 12824, 7736, 13058, 20764, 11734, 19896, 10608, 8444, 4918, 20520, 12870, 9006, 9110, 804, 24394, 3056, 82, 2072, 10096, 4722, 25086, 3044, 40858, 5258, 7670, 21726, 18530, 10632, 8096, 21288, 2734, 7918, 76, 34414, 10412, 2058, 1658, 15646, 10322, 19888, 5062, 470, 10602, 4046, 36824, 634, 10652, 2022, 12204, 3376, 25708, 8934, 918, 1266, 1326, 16916, 17628, 1906, 5500, 10112, 16868, 12422, 1130792, 81764, 73480, 7486, 3864, 19282, 4630, 2348, 8618, 230, 2580, 6810, 1364, 4866, 44608, 24, 15646, 832, 1248, 12112, 1954, 1976, 4236, 896, 6212, 4328, 3044, 3526, 7498, 3748, 6464, 912, 6230, 8524, 5246, 670, 68, 6726, 6466, 8476, 2742, 2494, 920, 54, 566, 1296, 6676, 6, 6226, 5422, 1712, 9070, 134, 6036, 296, 4112, 4604, 468, 2614, 272, 2680, 10248, 586, 458, 2070, 1388, 2222, 3612, 3328, 17158, 1494, 784, 1268, 568, 738, 10716, 366, 1070, 5912, 368, 4842, 866, 1738, 1208, 910, 1866, 236, 336, 1372, 4302, 564, 1086, 338, 4588, 2060, 1048, 2598, 5624, 102, 50, 7260, 408, 4062, 13520, 1702, 130, 1076, 2764, 802, 1180, 1228, 138, 9400, 3368, 1006, 288, 512, 28, 3002, 2396, 2338, 1132, 2900, 1866, 1278, 540, 1420, 7244, 3512, 104, 696, 3832, 1016, 2772, 156, 370, 40, 2986, 644, 3470, 536, 2630, 4488, 7282, 604, 314, 7258, 2326, 1070, 474, 4240, 168, 1132, 212, 2708, 2704, 11840, 1556, 8156, 19774, 538, 2298, 7166, 5152, 734, 912, 3964, 24074, 1840, 676, 320, 1282, 128, 434, 2026, 5166, 794, 1464, 13028, 834, 384, 2834, 3536, 32, 130, 10640, 7618, 8720, 554, 7684, 19642, 4992, 9952, 1874, 1112, 238, 1296, 3090, 44, 5044, 11520, 13700, 426, 15316, 7222, 3292, 10630, 13818, 790, 1716, 12410, 1224, 6596, 3250, 1902, 1150, 4070, 6904, 2934, 13978, 4424, 3822, 2364, 8160, 988, 3472, 11214, 3604, 962, 4884, 18392, 2568, 6966, 13010, 876, 8768, 1312, 662, 394, 4890, 5574, 4314, 23116, 9608, 7354, 28008, 16294, 4192, 3426, 5442, 12430, 4140, 310, 666, 11438, 3886, 1388, 2716, 770, 6892, 1460, 11504, 1516, 646, 6000, 3422, 4586, 256, 196, 1708, 12, 118, 1200, 3338, 1496, 936, 1692, 896, 18, 18, 684, 1796, 858, 5142, 5572, 3216, 9910, 804, 32, 98, 10, 846, 20, 1914, 3228, 132, 1386, 1432, 1680, 4372, 3650, 4286, 2088, 1036, 13968, 660, 1564, 3130, 9906, 12948, 728, 7188, 5778, 484, 5838, 16452, 6488, 1306, 1500, 5246, 2784, 68, 5100, 2462, 4778, 92, 1122, 3606, 126, 2994, 7470, 1752, 6988, 734, 9728, 784, 814, 7530, 2554, 2194, 8892, 3022, 1876, 1170, 7080, 1036, 698, 334, 964, 2892, 1830, 264, 11372, 976, 814, 1288, 5360, 2322, 594, 1344, 1106, 5314, 16796, 8886, 2054, 822, 3530, 510, 610, 10400, 8064, 1554, 2610, 4274, 9744, 3446, 2338, 2784, 526, 418, 924, 2262, 3508, 738, 19352, 2818, 94, 422, 2132, 22130, 228, 14, 1940, 1604, 346, 172, 4532, 3258, 6640, 8728, 11374, 5538, 4890, 2460, 3304, 9684, 1614, 212, 1668, 3224, 1216, 4036, 434, 4098, 250, 1934, 2436, 1124, 3206, 8744, 1694, 1872, 1996, 5700, 7598, 3386, 11792, 2080, 1272, 582, 872, 1230, 3068, 10202, 9354, 1826, 3594, 920, 1806, 2484, 6692, 384, 848, 2544, 626, 1630, 1042, 7934, 20, 2672, 372, 1880, 4320, 596, 3450, 5650, 2242, 382, 1902, 938, 192, 2776, 2118, 2006, 3672, 4754, 112, 2250, 11950, 468, 156, 4782, 1564, 660, 94, 42, 2334, 266, 160, 2298, 60, 790, 3472, 50, 186, 798, 580, 364, 1434, 42, 2676, 100, 2982, 66, 948, 436, 824, 1472, 1308, 30, 950, 124, 162, 310, 896, 1800, 2194, 102, 90, 54, 2754, 708, 3786, 202, 78, 34, 286, 44, 3188, 76, 3734, 718, 1966, 478, 3426, 2092, 4644, 4150, 4928, 3196, 570, 1614, 3614, 4058, 2090, 2398, 1142, 1916, 3100, 180, 14, 622, 534, 1360, 928, 2384, 4838, 934, 184, 4072, 2754, 4894, 768, 674, 1440, 1194, 3906, 336, 2298, 2732, 2630, 3242, 3906, 98, 1290, 2792, 3304, 422, 394, 3588, 2936, 2310, 886, 4560, 2668, 1754, 546, 8078, 2514, 11338, 622, 14, 1416, 2486, 1604, 2812, 1326, 6840, 2896, 508, 1246, 168, 1484, 1382, 348, 996, 506, 924, 142, 2150, 340, 4378, 2714, 2598, 702, 4956, 6172, 6, 1964, 3928, 1640, 22, 948, 22, 60, 9278, 2718, 8004, 170, 250, 3134, 2564, 1244, 2002, 2242, 54, 1740, 1208, 2590, 1822, 10318, 2976, 2918, 2456, 38, 490, 4006, 1260, 2692, 454, 1718, 3882, 20, 3544, 16, 2440, 1756, 30, 2076, 2610, 9430, 588, 2664, 7438, 1912, 4830, 1694, 3396, 466, 798, 2286, 3240, 3048, 3036, 3676, 1568, 1586, 2084, 1424, 418, 514, 960, 444, 82, 2776, 1470, 6, 1292, 3008, 188, 16, 10816, 976, 84, 14, 768, 3972, 104, 2516, 12, 208, 1700, 1354, 330, 1966, 1874, 1508, 2046, 3056, 20, 1304, 188, 2548, 9810, 4488, 2142, 3314, 6892, 1348, 2452, 2148, 32, 3070, 4270, 4760, 236, 1084, 518, 40, 4992, 3792, 5028, 200, 2056, 74, 110, 1290, 448, 3352, 2898, 2534, 44, 1942, 1084, 898, 4596, 4722, 2090, 1900, 334, 8, 194, 100, 2780, 2102, 38, 424, 904, 752, 4536, 1230, 140, 2660, 304, 2352, 2742, 6, 1040, 906, 238, 4020, 334, 1848, 4682, 356, 4634, 98, 5300, 392, 52, 608, 4276, 66, 2026, 888, 152, 466, 962, 3160, 4278, 100, 1814, 1882, 2230, 4484, 1316, 714, 44, 1570, 6314, 8, 292, 4380, 118, 466, 190, 166, 478, 1602, 1462, 808, 2960, 20, 3740, 1178, 222, 502, 3342, 4658, 10, 1846, 2124, 1608, 1068, 474, 2116, 106, 302, 3554, 574, 604, 488, 678, 32, 482, 1980, 2090, 30, 3800, 470, 3266, 2730, 1024, 94, 728, 740, 308, 534, 224, 874, 3200, 1188, 868, 2984, 3478, 1580, 1370, 662, 1452, 3310, 6, 1682, 12, 1174, 280, 2286, 772, 1472, 144, 280, 5066, 1822, 18, 128, 22, 3258, 110, 770, 212, 3516, 400, 294, 76, 11056, 210, 38, 3914, 158, 3154, 54, 2920, 468, 2876, 20, 1342, 3078, 94, 1058, 32, 250, 988, 1002, 768, 142, 1630, 338, 414, 4636, 128, 248, 84, 774, 218, 164, 3084, 1106, 714, 5110, 374, 22, 886, 20, 324, 174, 456, 28, 406, 596, 1188, 436, 1102, 230, 246, 746, 900, 1444, 14, 52, 580, 960, 770, 36, 6, 244, 1770, 1706, 522, 2270, 130, 3014, 92, 100, 12, 158, 1130, 1152, 2436, 14, 1106, 3144, 272, 26, 2640, 940, 556, 110, 2560, 1292, 30, 310, 666, 402, 578, 802, 46, 1518, 6, 1654, 120, 1410, 502, 1330, 486, 18, 132, 16, 560, 360, 78, 852, 730, 1144, 492, 2176, 1274, 110, 1704, 1384, 678, 604, 138, 354, 232, 50, 816, 1662, 32, 118, 8, 950, 128, 478, 150, 328, 122, 336, 102, 242, 2244, 468, 362, 664, 370, 230, 668, 78, 452, 622, 2316, 3394, 520, 170, 802, 240, 850, 110, 668, 1192, 3454, 382, 2058, 1648, 284, 630, 238, 528, 708, 1018, 2090, 428, 448, 622, 14, 404, 480, 164, 26, 802, 988, 206, 548, 500, 266, 366, 578, 396, 2524, 626, 1058, 742, 364, 168, 220, 1882, 1742, 430, 340, 64, 442, 3556, 706, 38, 294, 622, 288, 464, 120, 122, 860, 3572, 1904, 108, 660, 84, 468, 388, 1744, 92, 226, 50, 934, 88, 28, 252, 2234, 824, 712, 34, 450, 1274, 58, 218, 496, 884, 758, 910, 204, 204, 522, 134, 1020, 356, 110, 10, 142, 1508, 2194, 1288, 1024, 4626, 812, 1458, 606, 530, 728, 250, 860, 320, 394, 442, 616, 206, 440, 78, 422, 598, 1482, 124, 352, 678, 244, 334, 480, 396, 2334, 380, 414, 16, 20, 736, 918, 198, 80, 468, 2780, 84, 1572, 970, 856, 204, 2342, 186, 1622, 48, 24, 420, 550, 674, 42, 56, 284, 542, 88, 184, 642, 292, 738, 310, 26, 248, 1002, 470, 622, 422, 388, 664, 1638, 312, 74, 1300, 218, 98, 434, 444, 658, 202, 166, 636, 1200, 22, 8, 370, 814, 612, 1056, 32, 1532, 30, 258, 592, 42, 570, 1544, 86, 172, 1554, 838, 694, 508, 1322, 492, 602, 114, 166, 32, 332, 556, 406, 464, 176, 404, 878, 502, 140, 32, 138, 12, 1166, 640, 1434, 80, 102, 28, 10, 1122, 16, 40, 366, 10, 1646, 150, 518, 1098, 438, 52, 1194, 70, 274, 92, 2200, 308, 64, 560, 704, 230, 140, 624, 504, 440, 1024, 58, 142, 370, 1544, 1170, 466, 396, 44, 2302, 8, 100, 38, 162, 236, 226, 1174, 222, 424, 254, 324, 886, 92, 200, 156, 2120, 38, 2408, 702, 554, 42, 76, 20, 58, 200, 548, 494, 48, 540, 42, 1164, 632, 1292, 12, 94, 1240, 92, 468, 576, 72, 1518, 34, 140, 1148, 32, 2020, 126, 1834, 2186, 1686, 72, 28, 196, 734, 1486, 1272, 810, 2066, 596, 758, 238, 956, 312, 16, 500, 478, 132, 272, 76, 1590, 622, 1842, 210, 894, 12, 274, 130, 218, 876, 518, 600, 118, 14, 480, 152, 354, 994, 38, 1064, 954, 278, 144, 28, 734, 108, 908, 56, 562, 244, 204, 112, 68, 218, 528, 814, 1464, 688, 1436, 826, 378, 6, 3548, 104, 210, 6, 32, 1796, 570, 242, 632, 486, 60, 158, 272, 848, 1148, 148, 84, 338, 2286, 1350, 328, 2262, 556, 524, 1066, 796, 452, 218, 264, 186, 128, 704, 140, 12, 16, 288, 1570, 10, 190, 148, 242, 122, 1040, 514, 2994, 3054, 130, 30, 596, 196, 88, 110, 36, 3108, 216, 406, 718, 110, 600, 48, 50, 424, 2056, 366, 32, 22, 404, 3026, 1020, 582, 6, 216, 22, 1488, 1378, 2084, 722, 902, 64, 820, 364, 232, 150, 684, 620, 74, 308, 676, 40, 16, 1316, 2810, 28, 1432, 30, 60, 316, 1662, 872, 686, 588, 3240, 892, 966, 512, 576, 1420, 168, 602, 198, 216, 118, 370, 1762, 348, 166, 660, 202, 208, 392, 58, 1282, 254, 432, 20, 156, 14, 126, 54, 426, 2328, 960, 14, 108, 308, 58, 138, 1318, 44, 358, 146, 262, 14, 726, 2334, 406, 62, 1054, 1064, 74, 622, 98, 4034, 24, 534, 1144, 130, 448, 370, 88, 1486, 28, 558, 182, 540, 634, 1968, 1222, 1462, 66, 674, 258, 234, 434, 168, 18, 674, 1036, 522, 202, 6, 608, 72, 248, 144, 556, 2766, 14, 122, 1036, 334, 22, 54, 314, 610, 696, 458, 184, 62, 1786, 440, 1250, 684, 650, 628, 66, 34, 528, 666, 342, 1174, 1082, 374, 142, 134, 2094, 308, 712, 812, 1020, 488, 1006, 1010, 6, 92, 56, 48, 434, 238, 72, 194, 294, 792, 166, 90, 462, 396, 8, 90, 10, 2064, 60, 744, 36, 270, 920, 28, 254, 158, 688, 208, 552, 402, 1206, 930, 2174, 900, 8, 128, 248, 92, 76, 210, 156, 258, 2466, 148, 582, 10, 44, 350, 406, 360, 106, 410, 14, 596, 1268, 42, 324, 12, 84, 386, 482, 112, 10, 846, 338, 200, 462, 234, 192, 12, 52, 342, 72, 382, 72, 758, 482, 68, 842, 440, 24, 124, 6, 96, 522, 368, 572, 10, 118, 296, 348, 194, 148, 232, 244, 212, 70, 20, 180, 236, 42, 28, 36, 868, 50, 298, 28, 94, 344, 6, 44, 740, 138, 474, 416, 14, 370, 122, 600, 260, 516, 8, 522, 96, 574, 316, 108, 504, 186, 560, 44, 8, 16, 104, 348, 136, 194, 44, 188, 52, 214, 1032, 30, 356, 8, 656, 54, 140, 420, 200, 360, 738, 330, 244, 50, 106, 152, 448, 822, 134, 402, 808, 282, 1174, 360, 72, 124, 664, 620, 1938, 26, 20, 6, 528, 662, 14, 34, 144, 46, 260, 32, 536, 114, 382, 74, 960, 12, 718, 12, 32, 316, 100, 186, 550, 574, 12, 60, 642, 20, 430, 44, 66, 118, 206, 354, 134, 280, 224, 834, 900, 54, 718, 124, 80, 628, 238, 362, 668, 140, 106, 756, 98, 1120, 14, 10, 376, 170, 40, 534, 932, 6, 252, 14, 1268, 1050, 208, 132, 318, 1106, 200, 62, 518, 236, 518, 410, 524, 88, 448, 94, 736, 940, 36, 326, 28, 68, 346, 212, 508, 34, 148, 150, 54, 58, 688, 14, 108, 178, 332, 1776, 822, 258, 230, 42, 214, 12, 936, 10, 618, 488, 28, 172, 494, 232, 568, 580, 282, 230, 38, 1242, 48, 514, 62, 550, 34, 420, 786, 176, 110, 836, 6, 106, 768, 400, 444, 212, 36, 32, 126, 194, 420, 38, 114, 488, 68, 236, 304, 1200, 198, 58, 10, 64, 1016, 588, 958, 174, 188, 126, 50, 608, 302, 1278, 332, 106, 1048, 560, 190, 24, 6, 22, 322, 708, 130, 246, 222, 124, 166, 104, 176, 646, 20, 28, 732, 362, 80, 164, 150, 168, 1068, 368, 314, 190, 58, 230, 1192, 280, 172, 752, 330, 170, 364, 6, 76, 452, 304, 356, 6, 114, 616, 490, 202, 144, 14, 14, 594, 1000, 72, 10, 604, 102, 6, 242, 720, 6, 102, 116, 40, 100, 368, 662, 282, 640, 304, 40, 1908, 130, 480, 142, 6, 750, 46, 1022, 1164, 166, 36, 94, 30, 36, 234, 28, 164, 520, 572, 72, 110, 598, 24, 926, 396, 860, 248, 260, 442, 80, 84, 1016, 64, 156, 418, 588, 480, 296, 146, 82, 114, 254, 8, 252, 10, 350, 1054, 928, 172, 416, 316, 28, 74, 284, 412, 82, 80, 186, 1570, 316, 518, 90, 400, 14, 18, 406, 126, 1964, 426, 632, 22, 1064, 354, 468, 566, 42, 54, 374, 764, 546, 30, 36, 302, 220, 28, 108, 126, 20, 112, 16, 186, 94, 64, 148, 200, 58, 350, 208, 184, 240, 44, 116, 208, 8, 84, 108, 740, 1190, 82, 256, 140, 8, 34, 6, 426, 108, 274, 68, 530, 322, 646, 1228, 42, 426, 8, 8, 1106, 8, 48, 2446, 112, 52, 8, 200, 330, 604, 192, 632, 236, 56, 394, 406, 448, 146, 266, 70, 274, 82, 48, 38, 52, 232, 652, 560, 924, 426, 26, 144, 754, 24, 90, 158, 12, 284, 28, 516, 654, 272, 10, 78, 776, 984, 6, 8, 2216, 118, 6, 550, 106, 122, 424, 88, 330, 370, 688, 416, 1222, 20, 34, 188, 378, 542, 18, 1550, 172, 516, 82, 10, 480, 182, 66, 970, 52, 12, 812, 700, 44, 46, 10, 566, 8, 12, 778, 576, 48, 628, 356, 150, 102, 186, 1240, 238, 382, 244, 470, 8, 146, 100, 380, 514, 360, 1350, 24, 564, 304, 8, 236, 404, 288, 1784, 2158, 274, 964, 194, 1340, 164, 64, 84, 152, 166, 226, 394, 8, 618, 28, 768, 302, 800, 842, 432, 1074, 616, 228, 724, 934, 338, 640, 228, 428, 66, 88, 186, 116, 40, 6, 2240, 44, 148, 208, 352, 78, 166, 32, 572, 96, 1282, 110, 538, 374, 558, 476, 146, 228, 546, 1094, 298, 1868, 644, 8, 276, 20, 2960, 538, 12, 6, 728, 200, 92, 6, 926, 852, 252, 78, 48, 60, 6, 1866, 8, 1424, 372, 716, 82, 12, 1728, 1064, 1266, 100, 264, 6, 542, 10, 134, 428, 1056, 1328, 76, 152, 38, 436, 592, 178, 1110, 230, 102, 300, 598, 962, 490, 658, 44, 764, 284, 616, 90, 80, 228, 1274, 400, 574, 324, 34, 6, 320, 482, 76, 926, 6, 132, 6, 586, 20, 256, 2202, 360, 110, 48, 10, 338, 150, 22, 196, 634, 360, 288, 304, 1190, 72, 220, 1352, 304, 6, 104, 876, 20, 828, 462, 1202, 22, 118, 1700, 1776, 1004, 228, 264, 230, 58, 84, 2544, 60, 946, 436, 222, 162, 718, 136, 166, 420, 208, 248, 62, 1130, 70, 890, 62, 664, 1186, 22, 774, 14, 618, 132, 462, 18, 10, 26, 1342, 1030, 444, 38, 84, 176, 222, 418, 270, 364, 40, 12, 156, 288, 1576, 20, 652, 2514, 826, 356, 48, 446, 1822, 242, 450, 324, 98, 142, 496, 164, 8, 544, 28, 1084, 306, 2296, 236, 172, 50, 652, 778, 8, 62, 606, 192, 528, 892, 678, 70, 20, 158, 546, 1146, 2032, 634, 732, 1072, 8, 188, 900, 826, 1746, 528, 178, 6, 40, 6, 126, 848, 662, 434, 364, 590, 138, 6, 1188, 1404, 1138, 472, 538, 894, 64, 806, 96, 228, 408, 288, 584, 744, 1590, 64, 496, 562, 688, 2016, 382, 8, 6, 132, 716, 3364, 8, 20, 232, 8, 1152, 138, 2202, 192, 72, 446, 58, 1278, 28, 3892, 2212, 70, 1368, 1078, 184, 432, 564, 486, 104, 16, 1148, 416, 54, 400, 2256, 1314, 34, 980, 78, 28, 598, 798, 76, 102, 1680, 54, 50, 234, 322, 6, 104, 278, 438, 416, 6, 1098, 184, 684, 6, 356, 46, 688, 40, 6, 3876, 468, 628, 172, 1036, 330, 1170, 416, 42, 2068, 214, 518, 450, 46, 1012, 352, 2022, 904, 20, 704, 6, 62, 946, 1038, 18, 12, 502, 46, 256, 1506, 6, 216, 382, 144, 456, 142, 3028, 6, 2364, 154, 586, 732, 364, 2870, 774, 900, 1790, 1280, 124, 118, 1404, 716, 1174, 142, 52, 1220, 220, 464, 6, 2460, 830, 3100, 1668, 10, 1070, 8, 2356, 874, 330, 568, 508, 300, 994, 1404, 14, 34, 378, 138, 1022, 1140, 3372, 230, 802, 706, 234, 58, 386, 6, 1752, 342, 174, 30, 286, 14, 52, 190, 200, 30, 398, 6, 528, 6, 50, 954, 3738, 1282, 6, 6, 1238, 14, 1110, 640, 154, 1120, 2426, 90, 1374, 1930, 506, 404, 50, 86, 1772, 80, 230, 92, 1168, 358, 2296, 330, 6, 528, 6, 6, 528, 8, 82, 688, 8, 176, 6, 24, 1094, 6, 190, 90, 6, 10, 1644, 32, 18, 486, 1272, 12, 58, 18, 112, 1056, 132, 6, 846, 38, 916, 8, 14, 308, 32, 1294, 370, 1664, 258, 22, 540, 186, 944, 46, 22, 1000, 1246, 136, 6, 6, 166, 150, 90, 6, 290, 6, 616, 8, 18, 6, 396, 1246, 1538, 528, 516, 100, 6, 16, 170, 10, 96, 1114, 6, 606, 1370, 110, 12, 1458, 56, 576, 1142, 6, 6, 6, 294, 330, 916, 6, 588, 480, 6, 8, 64, 6, 598, 72, 686, 696, 330, 6, 626, 240, 1160, 198, 28, 320, 6, 1870, 6, 3606, 322, 296, 6, 8, 6, 6, 564, 1078, 54, 544, 1350, 350, 414, 6, 990, 6, 118, 50, 166, 6, 6, 1630, 6, 718, 2138, 6, 432, 6, 6, 6, 6, 6, 6, 6, 750, 1036, 368, 1386, 2850, 966, 576, 1364, 140, 1472, 6, 56, 6, 6, 484, 794, 6, 6, 278, 6, 6, 78, 6, 6, 6, 6, 38, 6, 120, 6, 328, 6, 1042, 6, 6, 628, 972, 6, 6, 6, 624, 6, 6, 6, 6, 990, 6, 6, 6, 6, 1370, 504, 6, 360, 858, 6, 6, 6, 26, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 634, 1044, 6, 6, 6, 6, 6, 6, 6, 6, 38, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 870, 6, 6, 6, 6, 584, 20, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 250, 6, 6, 6, 6, 1486, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 20, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 12, 6, 412, 582, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
Training took:  543.887330532074  seconds.
Mean of solution length:  721.8336
		 W	W	
		 W	W	
G	B	 Y	O	 B	G	 O	Y	
B	G	 O	Y	 G	B	 Y	O	
		 R	R	
		 R	R	
-------------------------------
Move:  Initial
		 W	O	
		 W	Y	
G	B	 Y	R	 B	G	 O	W	
B	G	 O	R	 G	B	 Y	W	
		 R	Y	
		 R	O	
-------------------------------
Move:  R
		 W	R	
		 W	R	
G	B	 Y	Y	 B	G	 O	O	
B	G	 O	O	 G	B	 Y	Y	
		 R	W	
		 R	W	
-------------------------------
Move:  R
		 W	R	
		 W	R	
Y	Y	 B	G	 O	O	 G	B	
B	G	 O	O	 G	B	 Y	Y	
		 R	W	
		 R	W	
-------------------------------
Move:  U
		 W	R	
		 W	R	
Y	Y	 B	G	 O	O	 G	B	
Y	Y	 B	G	 O	O	 G	B	
		 R	W	
		 R	W	
-------------------------------
Move:  Dprime
		 W	G	
		 W	G	
Y	Y	 B	W	 O	O	 G	R	
Y	Y	 B	W	 O	O	 G	R	
		 R	B	
		 R	B	
-------------------------------
Move:  R
		 W	W	
		 W	W	
Y	Y	 B	B	 O	O	 G	G	
Y	Y	 B	B	 O	O	 G	G	
		 R	R	
		 R	R	
-------------------------------
Move:  R'''

## Conclusions

Reinforcement Learning seems like a perfectly valid way to solve a Rubik's cube.  I would be interested in using a neural network as the Q value to see how that impacts performance because, while this works, it's a very slow process.  I was expecting to be able to migrate this information to create a 3x3 cube, but that didn't seem to be feasible as testing using 30 initial moves ended up taking 10-20 minutes per iteration.  

The process to change it to 3x3 would just involve changing the initial state to have three rows/columns instead of 2.  `validMoves` would actually be identical, because the accepted cube moves don't include a "middle" rotation, so it would still just be left, right, up, down and their primes.  Again, the possibility of adding a Front, Frontprime, Back, Backprime would greatly increase the possible Q table values greatly.

`makeMove` would get a little more complicated, but would largely remain the same.  

In [78]:
import io
from nbformat import current
import glob
nbfile = glob.glob('Edwards-Project.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file Edwards-Project.ipynb is 1455
